# DATA2001 Assignment

#First, we will begin by importing the various libraries.

In [1]:
## Importing the libraries
import pandas as pd
import matplotlib as plot
import numpy as np
import geopandas as gpd

In [2]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json
import os

data_path = "."

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        HOST       = db_conn_dict['host']
        DB_USER    = db_conn_dict['user']
        DB_PW      = db_conn_dict['password']
        DEFAULT_DB = db_conn_dict['user']

        try:
            db = create_engine('postgres+psycopg2://'+DB_USER+':'+DB_PW+'@'+HOST+'/'+DEFAULT_DB, echo=False)
            conn = db.connect()
            print('connected')
        except Exception as e:
            print("unable to connect to the database")
            print(e)
        return db,conn

credfilepath = os.path.join(data_path, "data2x01_db.json")
db, conn = pgconnect(credfilepath) #connect to pgAdmin

print(db.table_names())

unable to connect to the database
(psycopg2.OperationalError) FATAL:  too many connections for database "y21s1d2x01_scha6408"

(Background on this error at: http://sqlalche.me/e/13/e3q8)


UnboundLocalError: local variable 'conn' referenced before assignment

The next following steps involves converting the csv files into tables on pgAdmin.

1. Uploading the statistical areas data file

In [4]:
# if you want to reset the table
conn.execute("DROP TABLE IF EXISTS statisticalareas")

statistical_areas = """CREATE TABLE IF NOT EXISTS statisticalareas (
                         area_id      Integer PRIMARY KEY,
                         area_name VARCHAR(20),
                         parent_area_id Integer
                   )"""
conn.execute(statistical_areas)
print("Successfully, created table for statistical areas...")
statistical_areas = pd.read_csv('StatisticalAreas.csv')

table_name = "statisticalareas"
statistical_areas.to_sql(table_name, con=conn, if_exists='replace',index=False)
res = pd.read_sql_query("SELECT * FROM statisticalareas",conn)
res


Successfully, created table for statistical areas...


,area_id,area_name,parent_area_id
0,1,New South Wales,0
1,10,Greater Sydney,1
2,11,Rest of NSW,1
3,2,Victoria,0
4,20,Greater Melbourne,2
...,...,...,...
429,106,Hunter Valley exc Newcastle,11
430,11102,Lake Macquarie - West,111
431,111,Newcastle and Lake Macquarie,11
432,11402,Southern Highlands,114


2. Creating a table for the BusinessStats.csv

In [5]:
conn.execute("DROP TABLE IF EXISTS businessstats")

business_stats = """CREATE TABLE IF NOT EXISTS businessstats (
                         area_id   Integer PRIMARY KEY,
                         area_name VARCHAR(20),
                         number_of_businesses Integer,
                         accomodation_and_food_services Integer,
                         retail_trade Integer,
                         agriculture_forestry_and_fishing Integer,
                         health_care_and_social_assistance Integer,
                         public_administration_and_safety Integer,
                         transport_postal_and_warehousing Integer
                   )"""
conn.execute(business_stats)
print("Successfully, created table for business stats...")
business_stats_file = pd.read_csv('BusinessStats.csv')

table_name = "businessstats"
business_stats_file.to_sql(table_name, con=conn, if_exists='replace',index=False)

#Testing if table has been created
print ("Business Stats table created")
res = pd.read_sql_query('SELECT * FROM businessstats', conn)
res


Successfully, created table for business stats...
Business Stats table created


,area_id,area_name,number_of_businesses,accommodation_and_food_services,retail_trade,agriculture_forestry_and_fishing,health_care_and_social_assistance,public_administration_and_safety,transport_postal_and_warehousing
0,101021007,Braidwood,629,26,27,280,11,0,35
1,101021008,Karabar,326,7,10,8,11,0,43
2,101021009,Queanbeyan,724,52,47,11,56,3,77
3,101021010,Queanbeyan - East,580,16,23,4,12,0,57
4,101021011,Queanbeyan Region,1642,39,63,292,34,7,81
...,...,...,...,...,...,...,...,...,...
2296,901011001,Christmas Island,0,0,0,0,0,0,0
2297,901021002,Cocos (Keeling) Islands,7,3,0,0,0,0,0
2298,901031003,Jervis Bay,6,0,3,0,0,0,0
2299,901041004,Norfolk Island,0,0,0,0,0,0,0


3. Creating a table for the Neighbourhoods.csv

In [6]:
# Creating a table for neighbourhoods
conn.execute("DROP TABLE IF EXISTS neighbourhoods")

neighbourhoods = """CREATE TABLE IF NOT EXISTS neighbourhoods (
                         area_id   Integer PRIMARY KEY,
                         area_name VARCHAR(20),
                         land_area Integer,
                         population Integer,
                         number_of_dwellings Integer,
                         number_of_businesses Integer,
                         median_annual_household_income Integer,
                         avg_monthly_rent Integer
                   )"""
conn.execute(neighbourhoods)
print("Successfully, created table for neighbourhoods...")

neighbourhoods_file = pd.read_csv('Neighbourhoods.csv')

#Testing if table has been created
table_name = "neighbourhoods"
neighbourhoods_file.to_sql(table_name, con=conn, if_exists='replace',index=False)

print ("Neighbourhoods table created")
res = pd.read_sql_query('SELECT * FROM neighbourhoods', conn)
res

Successfully, created table for neighbourhoods...
Neighbourhoods table created


,area_id,area_name,land_area,population,number_of_dwellings,number_of_businesses,median_annual_household_income,avg_monthly_rent
0,102011028,Avoca Beach - Copacabana,643.8000,7590,2325,738.0,46996.0,1906.0
1,102011029,Box Head - MacMasters Beach,3208.6000,10986,3847,907.0,42621.0,1682.0
2,102011030,Calga - Kulnura,76795.1000,4841,1575,1102.0,42105.0,1182.0
3,102011031,Erina - Green Point,3379.3000,14237,4450,1666.0,43481.0,1595.0
4,102011032,Gosford - Springfield,1691.2000,19385,6373,2126.0,45972.0,1382.0
...,...,...,...,...,...,...,...,...
317,106011109,Cessnock Region,1570.4341,"7,931","3,281",673.0,73164.0,1080.0
318,106011113,Singleton Region,4067.2349,"4,919","2,055",698.0,87984.0,1000.0
319,111021218,Morisset - Cooranbong,330.5208,"14,959","6,298",1154.0,58084.0,1260.0
320,114021285,Hill Top - Colo Vale,174.3752,"6,025","2,249",400.0,81120.0,1512.0


Now, let's import the shape file

In [7]:
from geoalchemy2 import Geometry, WKTElement
from shapely.geometry import Point, Polygon, MultiPolygon

countries = gpd.read_file( os.path.join("/Users/sidath/Desktop/data2001-assignment", "RFSNSW_BFPL.shp") )

print(countries)
print(countries.dtypes)


        CATEGORY  SHAPE_LENG    SHAPE_AREA                     geometry
0              1    0.000017  5.392400e-12  POINT (149.11319 -33.05824)
1              1    0.000178  1.140005e-09  POINT (152.27536 -29.68316)
2              1    0.000890  4.950178e-08  POINT (152.14244 -29.68266)
3              1    0.000442  8.094091e-09  POINT (152.27579 -29.68259)
4              1    0.000890  4.950155e-08  POINT (151.99619 -29.68131)
...          ...         ...           ...                          ...
516628         2    0.000942  3.484545e-08  POINT (153.49994 -28.47663)
516629         2    0.000457  8.385178e-09  POINT (153.49998 -28.62864)
516630         2    0.001833  1.341083e-07  POINT (153.50010 -28.51679)
516631         2    0.012603  3.187411e-06  POINT (153.38733 -28.49906)
516632         2    0.000555  8.779539e-09  POINT (153.49997 -28.53430)

[516633 rows x 4 columns]
CATEGORY         int64
SHAPE_LENG     float64
SHAPE_AREA     float64
geometry      geometry
dtype: object


In [33]:
countries.head()

,CATEGORY,SHAPE_LENG,SHAPE_AREA,geometry
0,1,0.000017,5.392400e-12,POINT (149.11319 -33.05824)
1,1,0.000178,1.140005e-09,POINT (152.27536 -29.68316)
2,1,0.000890,4.950178e-08,POINT (152.14244 -29.68266)
3,1,0.000442,8.094091e-09,POINT (152.27579 -29.68259)
4,1,0.000890,4.950155e-08,POINT (151.99619 -29.68131)


Now let's connect this to a database.

In [34]:
def pgexecute( conn, sqlcmd, args=None, msg='', silent=False ):
    """ utility function to execute some SQL query statement
       can take optional arguments to fill in (dictionary)
       will print out on screen the result set of the query
       error and transaction handling built-in """
    retval = False
    result_set = None

    try:
        if args is None:
            result_set = conn.execute(sqlcmd).fetchall()
        else:
            result_set = conn.execute(sqlcmd, args).fetchall()

        if silent == False: 
            print("success: " + msg)
            for record in result_set:
                print(record)
        retval = True
    except Exception as e:
        if silent == False:
            print("db read error: ")
            print(e)
    return retval

def pgquery( conn, sqlcmd, args=None, silent=False ):
    """ utility function to execute some SQL query statement
    can take optional arguments to fill in (dictionary)
    will print out on screen the result set of the query
    error and transaction handling built-in """
    retdf = pd.DataFrame()
    retval = False
    try:
        if args is None:
            retdf = pd.read_sql_query(sqlcmd,conn)
        else:
            retdf = pd.read_sql_query(sqlcmd,conn,params=args)
        if silent == False:
            print(retdf.shape)
            print(retdf.to_string())
        retval = True
    except Exception as e:
        if silent == False:
            print("db read error: ")
            print(e)
    return retval,retdf

In [35]:
# lets just make sure we have PostGIS working on our connection
postgis_check = '''
SELECT PostGIS_Version();
'''

retval,retdf = pgquery(conn,postgis_check)
retdf

(1, 1)
                         postgis_version
0  3.0 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


,postgis_version
0,3.0 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [36]:
RFSNSW_BFPL = '''CREATE TABLE rfsnsw_bfpl (
                     category NUMERIC, 
                     shape_leng NUMERIC, 
                     shape_area NUMERIC, 
                     geom GEOMETRY(POINT,4326))''' 

conn.execute("DROP TABLE IF EXISTS world")
conn.execute(RFSNSW_BFPL)

In [48]:
# Use GeoAlchemy's WKTElement to create a geom with SRID
# NOTE :: THIS ONLY CHANGES POLYGON's to MULTI POLYGONS, IF YOU HAVE OTHER TYPES IN YOUR DATA, 
#     YOU WILL HAVE TO CONSULT THE GEOALCHEMY AND SHAPELY DOCUMENTATION ON HOW TO HANDLE THOSE
def create_wkt_element(geom,srid):
    if (geom.geom_type == 'Point'):
        geom = MultiPoint([geom])
    return WKTElement(geom.wkt, srid)

In [47]:
srid = 4326

# since we are altering data again, we should create another copy
countriesWkCpy = countries.copy()
countriesWkCpy['geom'] = countries['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))
#delete the old column before insert
countriesWkCpy = countriesWkCpy.drop(columns="geometry")
countriesWkCpy

NameError: name 'MultiPoint' is not defined

In [49]:
# Use 'dtype' to specify column's type
# For the geom column, we will use GeoAlchemy's type 'Geometry'
world_table_name = "world"
countriesWkCpy.to_sql(world_table_name, conn, if_exists='append', index=False, 
                         dtype={'geom': Geometry('MULTIPOINT', srid)})

AttributeError: 'GeometryDtype' object has no attribute 'base'